In [1]:
import pandas as pd
import numpy as np
import pyodbc
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import gensim
from collections import defaultdict
from gensim import corpora
from gensim import models
from gensim import similarities


In [ ]:
host = r''
dbSQL = ''
user = ''
pwd = ''
MC0 = '''ABS AH- ALM BC- BL- CA- CAE CHG COM CU- FP- FW- GA- GBL GPP GUE HVC LBU LGR LIT MWT PLT PND PRD PWS RS- SCR SD- SDR SPP STB SW- TIC TRF UDT WST'''.split(' ')
MC1 = '''ABS AF- AH- ALM BC- BL- CA- CAE CBA CCW CHG CND COM CU- ESP EXC FAT FG- FP- FW- GA- GUE GYP HVC LIT MWT PFC PJF PLT PRD RS- BCW SD- SDR SIS STB SW- TA- TIC TRF'''.split(' ')
MC2 = '''ABS AF- AH- ALM BC- BL- CA- CAE CBA CCW CHG CND COM CU- ESP EXC FAT FG- FP- FW- GA- GUE GYP HVC LIT MWT PFC PJF PLT PRD RS- BCW SD- SDR SIS STB SW- TA- TIC TRF'''.split(' ')
MC3 = '''ABS AF- AH- ALM BC- BL- CA- CAE CBA CCW CHG CND COM CU- ESP EXC FAT FG- FP- FW- GA- GUE GYP HVC LIT MWT PFC PJF PLT PRD RS- SCR SD- SDR SIS STB SW- TA- TIC TRF'''.split(' ')
#Consider adding MC00X in fron to all above in code os I don't need to do it later in teh code :)
#I will do this in an algorithm later by pulling in MaximoDW.dbo.Locations and grabbing DISTINCT SUBSTRING(LOCATION, 4, 8) but right now i'm lazy
where = ''
for l in MC0: where += f'OR LOCATION LIKE \'MC000{l}%\' '
for l in MC1: where += f'OR LOCATION LIKE \'MC001{l}%\' '
for l in MC2: where += f'OR LOCATION LIKE \'MC002{l}%\' '
for l in MC3: where += f'OR LOCATION LIKE \'MC003{l}%\' '
where = where[3:]
maximoSelect = f"""SELECT [

"""
woSelect= f"""SELECT [WONUM]
      ,[LOCATION]
      ,[DESCRIPTION]
  FROM [Generation].[agent].[MaximoWorkOrders]
  Where cast(STATUSDATE as date) > '01/01/2020'
"""  #and LOC = 'MC'
#  And ({where})"""

#getting data and then relableing maximo location to higher level equipment name
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+host+';DATABASE='+dbSQL+';UID='+user+';PWD='+ pwd)
WOdata = pd.read_sql(woSelect, cnxn)
#for key in MC0: WOdata.LOCATION[WOdata.LOCATION.str.startswith('MC000' + key)] = 'MC000' + MC0[MC0.index(key)]
#for key in MC1: WOdata.LOCATION[WOdata.LOCATION.str.startswith('MC001' + key)] = 'MC001' + MC1[MC1.index(key)]
#for key in MC2: WOdata.LOCATION[WOdata.LOCATION.str.startswith('MC002' + key)] = 'MC002' + MC2[MC2.index(key)]
#for key in MC3: WOdata.LOCATION[WOdata.LOCATION.str.startswith('MC003' + key)] = 'MC003' + MC3[MC3.index(key)]
WOdata = WOdata.drop(columns=['WONUM'])
WOdata

In [ ]:
WOdata['Edited1'] = [list() for x in range(len(WOdata.index))]
WOdata['Edited2'] = [list() for x in range(len(WOdata.index))]

def processText(text):
    #text_tokens = word_tokenize(text)
    text_tokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text)
    text_tokens = [word for word in text_tokens if not word in stop_words]  ##Remove Stop Words
    text_tokens = [word.lower() for word in text_tokens]
    #text_tokens = [ps.stem(word) for word in text_tokens] #Stem Words
    #text_tokens = [word for word in text_tokens if len(word) > 2] #Removing noise
    #text_tokens = [word for word in text_tokens if word.isalpha()] #Remove Punctuation
    #text_tokens = [ele for ele in text_tokens if ele.strip()] #remove empty spaces in list
    return text_tokens
#Applying Text Processing
WOdata['Edited1'] = WOdata['DESCRIPTION'].apply(processText)
WOdata

In [4]:
#creating a corpus out of our collection of items
from gensim import corpora
texts = WOdata['Edited1'].to_list()
frequency = defaultdict(int)
'''
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]
'''
dictionary = corpora.Dictionary(WOdata['Edited1'])
corpus = [dictionary.doc2bow(text) for text in WOdata['Edited1']]
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=16)

In [5]:
dictionary.save('testDict.cor')

In [6]:
from gensim.test.utils import datapath
from gensim import utils
import gensim.models

model = gensim.models.Word2Vec(sentences=WOdata['Edited1'].to_list())
wv = model.wv
#vec_king = model.wv['Coal']
for index, word in enumerate(wv.index_to_key):
    if index ==100:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/8132 is pm
word #1/8132 is 1
word #2/8132 is 2
word #3/8132 is weekly
word #4/8132 is unit
word #5/8132 is 3
word #6/8132 is 4
word #7/8132 is daily
word #8/8132 is monthly
word #9/8132 is oil
word #10/8132 is and
word #11/8132 is pump
word #12/8132 is inspections
word #13/8132 is air
word #14/8132 is check
word #15/8132 is checks
word #16/8132 is inspection
word #17/8132 is a
word #18/8132 is clean
word #19/8132 is needs
word #20/8132 is inspect
word #21/8132 is units
word #22/8132 is valve
word #23/8132 is to
word #24/8132 is on
word #25/8132 is lubrication
word #26/8132 is b
word #27/8132 is u
word #28/8132 is system
word #29/8132 is c
word #30/8132 is water
word #31/8132 is 0
word #32/8132 is conveyor
word #33/8132 is 12
word #34/8132 is coal
word #35/8132 is u1
word #36/8132 is grease
word #37/8132 is mill
word #38/8132 is ash
word #39/8132 is u3
word #40/8132 is fan
word #41/8132 is annual
word #42/8132 is tc1
word #43/8132 is leaking
word #44/8132 is change
word #45/8132

In [7]:
target = 'weekly'
#wv.most_similar(target)
good_count = 0
for t in texts:
    if (wv.wmdistance(target, t) < .8):
        good_count += 1
        print(wv.wmdistance(target, t) )
        print(t)
    if(good_count > 10):
        break


Training off Quest rather than Work Order Data

In [8]:

query1 = """select [Name], [DrawingTextAll]  from [Generation].[quest].[Drawings] WHERE DrawingTextAll IS NOT NULL"""
queryProperties ={"user": "", "password": "", "driver":"org.postgresql.Driver","spark.jars":"file:/home/postgresTest/postgresql-42.2.9.jar","spark.executor.extraClassPath":"/home/postgresTest/postgresql-42.2.9.jar","spark.driver.extraClassPath":"/home/postgresTest/postgresql-42.2.9.jar", 'partitionColumn':'TAGID_PI', 'lowerBound': '1100000000' , 'upperBound': '1510890005', 'numPartitions':'80', 'fetchsize':'100000'}
server = 'VMGENENG5' 
database = 'Generation' 
username = 'GENENG' 
password = 'U+RrPP"ks2]<#2{6'
cnxn = pyodbc.connect('DRIVER=/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.9.so.1.1;SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
#You may need to change the libmsodbcsql-17.8.s0.1.1 if we update it. Open a terminal cd out of /u01/ and start going down opt
global questData
questData = pd.read_sql(query1,cnxn)
print("Data size: ", questData.count())
cnxn.close()



/home/spark/.local/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Data size:  Name              54313
DrawingTextAll    54313
dtype: int64


In [ ]:
questData['Edited1'] = [list() for x in range(len(questData.index))]
questData['Edited1'] = questData['DrawingTextAll'].apply(processText)

questData

In [10]:
#creating a corpus out of our collection of items
from gensim import corpora
questTexts = questData['Edited1'].to_list()
frequency = defaultdict(int)
'''
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]
'''
dictionary = corpora.Dictionary(questData['Edited1'])
dictionary.filter_extremes()
corpus = [dictionary.doc2bow(text) for text in questData['Edited1']]
#lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=16)

In [11]:
from gensim.test.utils import datapath
from gensim import utils
import gensim.models

model = gensim.models.Word2Vec(sentences=questTexts)
wv = model.wv
#vec_king = model.wv['Coal']
for index, word in enumerate(wv.index_to_key):
    if index ==20:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/105549 is 2
word #1/105549 is 1
word #2/105549 is 4
word #3/105549 is ee
word #4/105549 is 3
word #5/105549 is 7
word #6/105549 is a
word #7/105549 is no
word #8/105549 is e
word #9/105549 is 0
word #10/105549 is 5
word #11/105549 is to
word #12/105549 is 6
word #13/105549 is 8
word #14/105549 is s
word #15/105549 is of
word #16/105549 is for
word #17/105549 is c
word #18/105549 is ae
word #19/105549 is f


In [13]:
target = 'fgd'
wv.most_similar(target)

[('retrofit', 0.7165215015411377),
 ('fgo', 0.7153223752975464),
 ('pgd', 0.6869065165519714),
 ('retrof', 0.6608300805091858),
 ('handung', 0.6589170694351196),
 ('reconnect', 0.6557502746582031),
 ('retpofit', 0.6391232013702393),
 ('fgod', 0.6194508075714111),
 ('egd', 0.6157587170600891),
 ('broan', 0.6097511649131775)]